<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#随机梯度下降" data-toc-modified-id="随机梯度下降-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>随机梯度下降</a></span></li><li><span><a href="#分类" data-toc-modified-id="分类-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>分类</a></span></li><li><span><a href="#回归" data-toc-modified-id="回归-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>回归</a></span></li><li><span><a href="#稀疏数据的随机梯度下降" data-toc-modified-id="稀疏数据的随机梯度下降-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>稀疏数据的随机梯度下降</a></span></li><li><span><a href="#复杂度" data-toc-modified-id="复杂度-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>复杂度</a></span></li><li><span><a href="#稀疏矩阵在Python科学计算中的实际意义" data-toc-modified-id="稀疏矩阵在Python科学计算中的实际意义-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>稀疏矩阵在Python科学计算中的实际意义</a></span><ul class="toc-item"><li><span><a href="#Compressed-Sparse-Row-Format-(CSR)" data-toc-modified-id="Compressed-Sparse-Row-Format-(CSR)-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Compressed Sparse Row Format (CSR)</a></span></li></ul></li><li><span><a href="#实用小贴士" data-toc-modified-id="实用小贴士-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>实用小贴士</a></span></li></ul></div>

# 随机梯度下降

**随机梯度下降(SGD)** 是一种简单但又非常高效的方法，主要用于凸损失函数下线性分类器的判别式学习，例如(线性) 支持向量机 和 Logistic 回归 。 尽管 SGD 在机器学习社区已经存在了很长时间, 但是最近在 large-scale learning （大规模学习）方面 SGD 获得了相当大的关注。

SGD 已成功应用于在文本分类和自然语言处理中经常遇到的**大规模和稀疏**的机器学习问题。对于稀疏数据，本模块的分类器可以轻易的处理超过 10^5 的训练样本和超过 10^5 的特征。

Stochastic Gradient Descent （随机梯度下降法）的优势:

- 高效。
- 易于实现 (有大量优化代码的机会)。

Stochastic Gradient Descent （随机梯度下降法）的劣势:

- SGD 需要一些超参数，例如 regularization （正则化）参数和 number of iterations （迭代次数）。
- SGD 对 feature scaling （特征缩放）敏感。

`SGDClassifier`  
sklearn.linear_model.SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=None, tol=None, shuffle=True, verbose=0, epsilon=0.1, n_jobs=1, random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, class_weight=None, warm_start=False, average=False, n_iter=None)

# 分类

SGDClassifier 类实现了一个简单的随机梯度下降学习例程, 支持不同的 loss functions（损失函数）和 penalties for classification（分类处罚）。

作为另一个 classifier （分类器）, 拟合 SGD 我们需要两个 array （数组）：保存训练样本的 size 为 [n_samples, n_features] 的数组 X 以及保存训练样本目标值（类标签）的 size 为 [n_samples] 的数组 Y

In [6]:
from sklearn.linear_model import SGDClassifier

x = [[0., 0.],[1., 1.]]
y = [0, 1]

clf = SGDClassifier(loss='hinge', penalty='l2')
clf.fit(x,y)

g:\work\tf\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [8]:
clf.predict([[2,2]])

array([1])

In [9]:
clf.coef_

array([[ 9.91080278,  9.91080278]])

In [10]:
# 成员 intercept_ 保存 intercept（截距） （又称作 offset（偏移）或 bias（偏差））:
# 模型是否使用 intercept（截距）, 即 a biased hyperplane(一个偏置的超平面), 是由参数 fit_intercept 控制的。
clf.intercept_

array([-9.99002993])

In [11]:
# 使用 SGDClassifier.decision_function 来获得到此超平面的 signed distance (符号距离)
clf.decision_function([[2,2]])

array([ 29.65318117])

具体的 loss function（损失函数） 可以通过 loss 参数来设置。 SGDClassifier 支持以下的 loss functions（损失函数）：

- loss="hinge": (soft-margin) linear Support Vector Machine （（软-间隔）线性支持向量机），
- loss="modified_huber": smoothed hinge loss （平滑的 hinge 损失），
- loss="log": logistic regression （logistic 回归），
- and all regression losses below（以及所有的回归损失）。

前两个 loss functions（损失函数）是懒惰的，如果一个例子违反了 margin constraint（边界约束），它们仅更新模型的参数, 这使得训练非常有效率,即使使用了 L2 penalty（惩罚）我们仍然可能得到稀疏的模型结果。

使用 loss="log" 或者 loss="modified_huber" 来启用 predict_proba 方法, 其给出每个样本 x 的概率估计 P(y|x) 的一个向量：

In [12]:
clf = SGDClassifier(loss="log").fit(x, y)
clf.predict_proba([[1., 1.]])     

g:\work\tf\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


array([[  4.97248476e-07,   9.99999503e-01]])

具体的惩罚方法可以通过 penalty 参数来设定。 SGD 支持以下 penalties（惩罚）:

- penalty="l2": L2 norm penalty on coef_.
- penalty="l1": L1 norm penalty on coef_.
- penalty="elasticnet": Convex combination of L2 and L1（L2 型和 L1 型的凸组合）; (1 - l1_ratio) * L2 + l1_ratio * L1.

默认设置为 penalty="l2" 。 L1 penalty （惩罚）导致稀疏解，使得大多数系数为零。 Elastic Net（弹性网）解决了在特征高相关时 L1 penalty（惩罚）的一些不足。参数 l1_ratio 控制了 L1 和 L2 penalty（惩罚）的 convex combination （凸组合）。

SGDClassifier 通过利用 “one versus all” （OVA）方法来组合多个二分类器，从而实现多分类。对于每一个 K 类, 可以训练一个二分类器来区分自身和其他 K-1 个类。在测试阶段，我们计算每个分类器的 confidence score（置信度分数）（也就是与超平面的距离），并选择置信度最高的分类。下图阐释了基于 iris（鸢尾花）数据集上的 OVA 方法。虚线表示三个 OVA 分类器; 不同背景色代表由三个分类器产生的决策面。

在 multi-class classification （多类分类）的情况下， coef_ 是 shape=[n_classes, n_features] 的一个二维数组， intercept_ 是 shape=[n_classes] 的一个一维数组。 coef_ 的第 i 行保存了第 i 类的 OVA 分类器的权重向量；类以升序索引 （参照属性 classes_ ）。 注意，原则上，由于它们允许创建一个概率模型，所以 loss="log" 和 loss="modified_huber" 更适合于 one-vs-all 分类。

SGDClassifier 通过拟合参数 class_weight 和 sample_weight 来支持 weighted classes （加权类）和 weighted instances（加权实例）。

SGDClassifier 支持 averaged SGD (ASGD)。Averaging（均值化）可以通过设置 `average=True` 来启用。AGSD 工作原理是在普通 SGD 的基础上，对每个样本的每次迭代后的系数取均值。当使用 ASGD 时，学习速率可以更大甚至是恒定，在一些数据集上能够加速训练过程。

对于带 logistic loss（logistic 损失）的分类，在 LogisticRegression 中提供了另一个采取 averaging strategy（平均策略）的 SGD 变体，其使用了随机平均梯度 (SAG) 算法。

# 回归

SGDRegressor 类实现了一个简单的随机梯度下降学习例程，它支持用不同的损失函数和惩罚来拟合线性回归模型。 SGDRegressor 非常适用于有大量训练样本（>10.000)的回归问题，对于其他问题，我们推荐使用 Ridge ，Lasso ，或 ElasticNet 。

具体的损失函数可以通过 loss 参数设置。 SGDRegressor 支持以下的损失函数:

loss="squared_loss": Ordinary least squares（普通最小二乘法）,
loss="huber": Huber loss for robust regression（Huber回归）,
loss="epsilon_insensitive": linear Support Vector Regression（线性支持向量回归）.
Huber 和 epsilon-insensitive 损失函数可用于 robust regression（鲁棒回归）。不敏感区域的宽度必须通过参数 epsilon 来设定。这个参数取决于目标变量的规模。

SGDRegressor 支持 ASGD（平均随机梯度下降） 作为 SGDClassifier。 均值化可以通过设置 `average=True` 来启用。

对于利用了 squared loss（平方损失）和 l2 penalty（l2惩罚）的回归，在 Ridge 中提供了另一个采取 averaging strategy（平均策略）的 SGD 变体，其使用了随机平均梯度 (SAG) 算法。

`SGDRegressor`  
sklearn.linear_model.SGDRegressor(loss='squared_loss', penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=None, tol=None, shuffle=True, verbose=0, epsilon=0.1, random_state=None, learning_rate='invscaling', eta0=0.01, power_t=0.25, warm_start=False, average=False, n_iter=None)

# 稀疏数据的随机梯度下降

在 scipy.sparse 支持的格式中，任意矩阵都有对稀疏数据的内置支持方法。但是，为了获得最高的效率，请使用 scipy.sparse.csr_matrix 中定义的 CSR 矩阵格式.

# 复杂度

SGD 主要的优势在于它的高效性，对于不同规模的训练样本，处理复杂度基本上是线性的。假如 X 是 size 为 (n, p) 的矩阵，训练成本为 O(k n \bar p)，其中 k 是迭代次数， \bar p 是每个样本非零特征的平均数。

但是，最近的理论结果表明，得到期望优化精度的运行时间并不会随着训练集规模扩大而增加。

# 稀疏矩阵在Python科学计算中的实际意义

对于那些零元素数目远远多于非零元素数目，并且非零元素的分布没有规律的矩阵称为稀疏矩阵（sparse）。

由于稀疏矩阵中非零元素较少，零元素较多，因此可以采用只存储非零元素的方法来进行压缩存储。对于一个用二维数组存储的稀疏矩阵Amn，如果假设存储每个数组元素需要L个字节，那么存储整个矩阵需要m*n*L个字节。但是，这些存储空间的大部分存放的是0元素，从而造成大量的空间浪费。**为了节省存储空间，可以只存储其中的非0元素。大大减少了空间的存储。**

另外对于很多元素为零的稀疏矩阵，仅存储非零元素可使矩阵操作效率更高。也就是稀疏矩阵的计算速度更快，因为只对非零元素进行操作，这是稀疏矩阵的一个突出的优点。

python不能自动创建稀疏矩阵，所以要用scipy中特殊的命令来得到稀疏矩阵。

## Compressed Sparse Row Format (CSR)
压缩稀疏行格式(CSR)通过四个数组确定： values,columns, pointerB, pointerE.

其中

数组values：是一个实（复）数，包含矩阵A中的非0元，以行优先的形式保存；

数组columns：第i个整型元素代表矩阵A中第i列；

数组pointerB ：第j个整型元素给出矩阵A行j中第一个非0元的位置，等价于pointerB(j) -pointerB(1)+1 ；

数组pointerE：第j个整型元素给出矩阵A第j行最后一个非0元的位置，等价于pointerE(j)-pointerB(1)。


```python
scipy.sparse.csr_matrix

class scipy.sparse.csr_matrix(arg1, shape=None, dtype=None, copy=False)[source]
    Compressed Sparse Row matrix
    This can be instantiated in several ways:
        csr_matrix(D)
            with a dense matrix or rank-2 ndarray D
        csr_matrix(S)
            with another sparse matrix S (equivalent to S.tocsr())
        csr_matrix((M, N), [dtype])
            to construct an empty matrix with shape (M, N) dtype is optional, defaulting to dtype=’d’.
        csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
            where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k].
        csr_matrix((data, indices, indptr), [shape=(M, N)])
            is the standard CSR representation where the column indices for row i are stored in indices[indptr[i]:indptr[i+1]] and their corresponding values are stored in data[indptr[i]:indptr[i+1]]. If the shape parameter is not supplied, the matrix dimensions are inferred from the index arrays.

```

# 实用小贴士

- **归一化，标准化** 随机梯度下降法对 feature scaling （特征缩放）很敏感，因此强烈建议您缩放您的数据。例如，将输入向量 X 上的每个特征缩放到 [0,1] 或 [- 1，+1]， 或将其标准化，使其均值为 0，方差为 1。请注意，必须将 相同 的缩放应用于对应的测试向量中，以获得有意义的结果。使用 StandardScaler: 很容易做到这一点：

```python
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)  # Don't cheat - fit only on training data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)  # apply same transformation to test data
```

> from sklearn.preprocessing import StandardScaler scaler = StandardScaler() scaler.fit(X_train) # Don’t cheat - fit only on training data X_train = scaler.transform(X_train) X_test = scaler.transform(X_test) # apply same transformation to test data

> 假如你的 attributes （属性）有一个固有尺度（例如 word frequencies （词频）或 indicator features（指标特征））就不需要缩放。

- **超参α** 最好使用 GridSearchCV 找到一个合理的 regularization term （正则化项） \alpha ， 它的范围通常在 10.0**-np.arange(1,7) 。
- **迭代次数** 经验表明，SGD 在处理约 10^6 训练样本后基本收敛。因此，对于迭代次数第一个合理的猜想是 n_iter = np.ceil(10**6 / n)，其中 n 是训练集的大小。
- ** **假如将 SGD 应用于使用 PCA 做特征提取，我们发现通过某个常数 c 来缩放特征值是明智的，比如使训练数据的 L2 norm 平均值为 1。
- 我们发现，当特征很多或 eta0 很大时， ASGD（平均随机梯度下降） 效果更好。

In [19]:
import numpy as np
    
n = 2342
n_iter = np.ceil(10**6 / n)

n_iter

427.0